This code serves for the 5th homework in Computational Neuroscicence taught by Xiao-Jing Wang in Fall 2022 semester. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import copy

In [ ]:
# generate N-dimensional input data with zero mean and some set covariance
def n_covariance(N,eta,K):
    # create A N*N matrix where each entry is drawn independently from a Gaussian distribution
    A = np.random.normal(0,np.sqrt(eta**2/N),size=(N,N))
    # covariance matrix
    Sigma = np.matmul(A,A.T)
    # epsilon, generate from N(0,1), generate K samples, K large
    xxset = np.zeros(shape=(K,N))
    for i in range(K):
        epsi = np.random.normal(0,1,size=(N,1))
        xx = np.matmul(A,epsi)
        xxset[i] = xx.T
        
    return [A,Sigma,xxset]

In [ ]:
# Problem 1
np.random.seed()

N = 10
eta = 1
K = 100 # how many data points
[A,Sigma,xx] = n_covariance(N,eta,K)

# Compute eigenvalues and eigenvectors
# Sigma = np.cov(xx.T)
[eigu,eigv] = np.linalg.eig(Sigma)

# largest eigenvalue
ind = np.argmax(eigu)
# corresponding eigenvector
lag_eigv = eigv[:,ind]

# print(f"eigenvalue:{eigu}")
# print(f"eigenvector:{eigv}")

# Initialize the weights at random from zero mean unit variance gaussian
W_oja = np.random.normal(0,1,size=(N,1))
prev_W_oja = np.ones((N,1))

learn_rate = 0.001
train_iteration = 1000
W_oja_norm = [] # norm
W_oja_set = [] # store
cnt = 0 # iterator

a1 = max(xx[:,0])/lag_eigv[0]
a2 = min(xx[:,0])/lag_eigv[0]

for i in range(train_iteration):
    Ys = np.dot(xx,W_oja)
    W_oja += learn_rate * np.sum(Ys*xx-np.square(Ys)*W_oja.T,axis=0).reshape((N,1))
    W_oja_norm.append(np.linalg.norm(W_oja))
    W_oja_set.append(W_oja.tolist())
    cnt += 1

    if cnt % 20 == 0 and cnt < 200:
        plt.figure(figsize=(5,5))
        plt.scatter(xx[:,0],xx[:,1],color='y',label='data pts')
        plt.plot([a1*lag_eigv[0],a2*lag_eigv[0]],[a1*lag_eigv[1],a2*lag_eigv[1]],color='r',label='PC1')
        plt.scatter(W_oja[0],W_oja[1],color='b',label='proj1')
        plt.title(f"Result when iteration={cnt}")

W_oja_set = np.array(W_oja_set)



plt.figure(figsize=(5,5))
plt.scatter(xx[:,0],xx[:,1],color='y',label='data pts')
plt.plot([a1*lag_eigv[0],a2*lag_eigv[0]],[a1*lag_eigv[1],a2*lag_eigv[1]],color='r',label='PC1')
plt.scatter(W_oja[0],W_oja[1],color='b',label='proj1')
plt.title(f"Result when iteration={cnt}")

plt.legend()
plt.title(f'Result when iteration={cnt}')
plt.show()

print(f"Required iteration:{cnt}")
print(f"W_oja after train:{W_oja}")
print(f"large eigenvector:{lag_eigv}")

plt.figure(figsize=(5,5)) 
plt.plot(W_oja_norm)
plt.xlabel('Iteration')
plt.ylabel('Norm')

if lag_eigv[0] * W_oja[0] <= 0:
    lag_eigv = -1 * lag_eigv
    
plt.figure(figsize=(5,5))
plt.scatter(W_oja_set[:,0],W_oja_set[:,1],label='trajectory of weights')
plt.scatter(lag_eigv[0],lag_eigv[1],label='First PC weight')
plt.xlabel('Weight 1')
plt.ylabel('Weight 2')
plt.legend()

plt.show()

time.sleep(1000)

In [ ]:
import time

# Problem 2
A_plus = 1.03
A_minus = -0.51
tau_plus = 14 # ms
tau_minus = 34 # ms
pre_freq = 10 / 10**3 # cycle/ms (10^3 times original Hz)
post_freq_range = np.linspace(0,25,50) / 10**3

# generate poisson process
def poisson_process(rate,tmax,bin_size):
    nbins = np.floor(tmax/bin_size).astype(int)
    prob_of_spike = rate * bin_size
    spikes = np.random.rand(nbins) < prob_of_spike
    return spikes * 1
    
def closest_neighbor(spike_f,spike_s):
    indx_f = np.where(spike_f == 1)[0]
    indx_s = np.where(spike_s == 1)[0]
    be_af_set = [] # length equal to number of presynaptic spike in duration
    for i in indx_f:
        try:
            largest_small = max(j for j in indx_s if j < i)
        except ValueError:
            largest_small = 1e30
        try:
            smallest_large = min(j for j in indx_s if j > i)
        except ValueError:
            smallest_large = 1e30

        temp_set = [largest_small,smallest_large]
        be_af_set.append(temp_set)
    return [indx_f,indx_s,be_af_set]

bin_size = 0.01 # ms
sim_time = 10000 # ms
repeat = 1000 # independent trails
actual_mean = []

for kkkk in range(len(post_freq_range)):
    across_trail = []
    for j in range(repeat):
        post_freq = post_freq_range[kkkk]
        pre_spikes_poisson = poisson_process(pre_freq,sim_time,bin_size)
        # print(f"number of presynaptic spikes: {sum(pre_spikes_poisson)}")
        post_spikes_poisson = poisson_process(post_freq,sim_time,bin_size)
        tt = np.arange(len(post_spikes_poisson)) * bin_size # time

        [indx_f,indx_s,be_af_set] = closest_neighbor(pre_spikes_poisson,post_spikes_poisson)
        deltaW_set = []

        for i in range(len(indx_f)):
            pretime = indx_f[i] * bin_size
            posttime_f = be_af_set[i][0] * bin_size
            posttime_s = be_af_set[i][1] * bin_size
            
            deltaW_f = 0
            deltaW_s = 0
            deltaW = 0
            # first postsynaptic event (before presynaptic)
            # t_post < t_pre
            if posttime_f < 1e8:
                deltaW_f = A_minus*np.exp(-(pretime-posttime_f)/tau_minus)
            # second postsynaptic event (before presynaptic)
            # t_post > t_pre
            if posttime_s < 1e8:
                deltaW_s = A_plus*np.exp((pretime-posttime_s)/tau_plus)
            deltaW += (deltaW_f + deltaW_s)
            deltaW_set.append(deltaW)
        # average across every synaptic event
        mean_deltaW = np.mean(deltaW_set)
        across_trail.append(mean_deltaW)
        # print(mean_deltaW)
    # average across different independent trails
    across_trail_mean = np.mean(across_trail)
    actual_mean.append(across_trail_mean)
    # print(across_trail_mean)

actual_mean = np.array(actual_mean)

stand_curve = []
for x in post_freq_range:
    stand_curve.append(x*(A_plus/(tau_plus**(-1)+x)+A_minus/(tau_minus**(-1)+x)))
stand_curve = np.array(stand_curve)

plt.figure(figsize=(5,5))
plt.scatter(post_freq_range*1000,actual_mean*100,color='r',label='experimental results')
plt.plot(post_freq_range*1000,stand_curve*100,color='g',label='standard curve')
plt.xlabel('Postsynaptic firing rate x, Hz')
plt.ylabel('Synaptic Change, %')
plt.legend()
plt.show()
